# Data

In [ ]:
#!value --name exampledata
125 17

125 17

In [ ]:
#!value --name data
554735 45401 8434 0 188 7487525 77 7

554735 45401 8434 0 188 7487525 77 7

# Utilities

In [ ]:
void Print(object s) {
    Console.WriteLine(s);
}

public static string DebugPrint<T>(this IEnumerable<T> self) =>
        new StringBuilder("[")
            .AppendJoin(", ", self)
            .Append(']')
            .ToString();

### Data Selector

In [ ]:
#!set --name fullData --value @value:data
#!set --name partialData --value @value:exampledata

var rawdata = fullData;
var data = rawdata.ReplaceLineEndings("\n");

### Imports

In [ ]:
using System;
using System.Collections;
using System.Text.RegularExpressions;
using System.Linq;
using System.Diagnostics;

# Part 1

In [ ]:
List<Int128> startStones = data.Split(" ").Select(s => Int128.Parse(s)).ToList();

startStones

index value 0 554735 1 45401 2 8434 3 0 4 188 5 7487525 6 77 7 7

In [ ]:
int StringLength(Int128 n) => $"{n}".Length;

bool IsEvenLength(Int128 n) => StringLength(n) % 2 == 0;

(Int128 s, Int128 e) LexicalSplit(Int128 n) {
    string nStr = $"{n}";
    int mid = nStr.Length / 2;
    string start = nStr.Substring(0, mid);
    string end = nStr.Substring(mid);
    return (Int128.Parse(start), Int128.Parse(end));
}

In [ ]:
List<Int128> Tick(List<Int128> state) {
    List<Int128> newState = [];
    foreach (Int128 stone in state) {
        if (stone == 0) {
            newState.Add(1);
        } else if (IsEvenLength(stone)) {
            var (start, end) = LexicalSplit(stone);
            newState.Add(start);
            newState.Add(end);
        } else {
            newState.Add(stone * 2024);
        }
    }
    return newState;
}

In [ ]:
List<Int128> TickN(List<Int128> startState, int n) {
    List<Int128> state = startState;
    var timer = Stopwatch.StartNew();
    var splits = Stopwatch.StartNew();
    for (int i = 0; i < n; i++) {
        state = Tick(state);
        // Print($"{i}: {timer.Elapsed}  {splits.Elapsed}");
        splits.Restart();
    }
    return state;
}

TickN(startStones, 25).Count

209412

# Part 2

In [ ]:
int[] rules = [0, 0, 0];

List<Int128> Tick2(List<Int128> state) {
    List<Int128> newState = [];
    foreach (Int128 stone in state) {
        if (stone == 0) {
            rules[0]++;
            newState.Add(1);
        } else if (IsEvenLength(stone)) {
            rules[1]++;
            var (start, end) = LexicalSplit(stone);
            newState.Add(start);
            newState.Add(end);
        } else {
            rules[2]++;
            newState.Add(stone * 2024);
        }
    }
    return newState;
}

In [ ]:
List<Int128> Tick2N(List<Int128> startState, int n) {
    List<Int128> state = startState;
    var timer = Stopwatch.StartNew();
    var splits = Stopwatch.StartNew();
    for (int i = 0; i < n; i++) {
        state = Tick2(state);
        // Print($"{i}: {timer.Elapsed}  {splits.Elapsed}");
        splits.Restart();
    }
    return state;
}

rules = [0, 0, 0];
display(Tick2N([0, 0], 25).Count);
display(rules);

39556

[ 3490, 39554, 30440 ]

In [ ]:
public static Int128 Sum(this IEnumerable<Int128> self) {
    Int128 acc = 0;
    foreach (var i in self) {
        acc += i;
    }
    return acc;
}

In [ ]:
// Build a lookup table mapping value and number of requested ticks to size of list.

Dictionary<(Int128 Value, int ticks), Int128> Solved = [];


Int128 ProcessValue(Int128 value, int ticks) {
    if (ticks == 0) {
        return 1;
    }
    (Int128 Value, int ticks) key = (value, ticks);
    if (!Solved.ContainsKey(key)) {
        var nextList = Tick([value]);
        Solved.Add(key, nextList.Select(v => ProcessValue(v, ticks - 1)).Sum());
    }
    return Solved[key];
}

Int128 ProcessList(List<Int128> list, int ticks) =>
    list.Select(v => ProcessValue(v, ticks)).Sum();

ProcessList(startStones, 75)

248967696501656